In [8]:
import sys

sys.path.append('../aml_pipeline/chuck_caption_component/')
# enhanced_doc_analyzer_component/enhanced_document_analyzer


from openai import AzureOpenAI


from src import DocumentProcessor

In [9]:
# Example of registering the component in a workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Get workspace
ml_client = MLClient.from_config(
    credential=DefaultAzureCredential()
)

Found the config file in: /config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [10]:
import dotenv
import os
import pandas as pd

# Load the environment variables
dotenv.load_dotenv()

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")

# Initialize the OpenAI client
openai_vision_client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint, 
    api_key=azure_openai_key, 
    api_version='2023-03-15-preview'
    )


In [18]:
# Initialize document processor
processor = DocumentProcessor(
    input_folder="./output-azure/0.55/",
    output_folder="./output-azure-chucked/1-s2.0-S0927796X2030053X-am.pdf",
    openai_client=openai_vision_client,
    vision_deployment_name="gpt-4v",
    max_chunk_length=512
)

stats, chunks_df = processor.process()

Image file does not exist: output-azure/0.55/output-azure/0.55/1-s2.0-S0927796X2030053X-am.pdf/elements/picture/page_3_Picture_140002005117824.png
Skipping caption generation for invalid image: output-azure/0.55/output-azure/0.55/1-s2.0-S0927796X2030053X-am.pdf/elements/picture/page_3_Picture_140002005117824.png
Image file does not exist: output-azure/0.55/output-azure/0.55/1-s2.0-S0927796X2030053X-am.pdf/elements/caption/page_3_Caption_140002005117344.png
Skipping caption generation for invalid image: output-azure/0.55/output-azure/0.55/1-s2.0-S0927796X2030053X-am.pdf/elements/caption/page_3_Caption_140002005117344.png
Image file does not exist: output-azure/0.55/output-azure/0.55/1-s2.0-S0927796X2030053X-am.pdf/elements/picture/page_6_Picture_140002032123760.png
Skipping caption generation for invalid image: output-azure/0.55/output-azure/0.55/1-s2.0-S0927796X2030053X-am.pdf/elements/picture/page_6_Picture_140002032123760.png
Image file does not exist: output-azure/0.55/output-azure/

In [12]:
stats

{'processed': 311,
 'errors': 0,
 'chunks': 412,
 'input_file': 'output-azure/0.55/1-s2.0-S0927796X2030053X-am.pdf/paper_elements.csv',
 'total_rows': 311,
 'timestamp': '2025-02-10T14:05:55.715121',
 'completion_status': 'success'}

In [13]:
chunks_df

'output-azure-chucked/1-s2.0-S0927796X2030053X-am.pdf/processed_chunks.jsonl'

In [14]:
from retriever_utils import ImageCaptionViewer

from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
import pandas as pd

azure_search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
azure_search_key = os.getenv("AZURE_SEARCH_KEY")
index_name="myindex"


# Initialize your search client
search_client = SearchClient(
    endpoint=azure_search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(azure_search_key)
)

# Get image chunks
results = search_client.search(
    search_text="*",
    filter="type eq 'image'",
    select=["id", "pdf_file", "page", "bounding_box", "text", "image_path", 
           "role", "confidence", "source"],
    top=1000
)

# Convert to DataFrame
chunks = []
for result in results:
    chunks.append(dict(result))
df = pd.DataFrame(chunks)

# # Create and display the viewer
viewer = ImageCaptionViewer(df)
viewer.display()

In [15]:
df.columns


Index(['source', 'bounding_box', 'text', 'id', 'role', 'image_path',
       'confidence', 'page', 'pdf_file', '@search.score',
       '@search.reranker_score', '@search.highlights', '@search.captions'],
      dtype='object')

In [16]:
for result in results:
    print(result)